In [9]:
from scipy.io import loadmat, savemat
import numpy as np
import time
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn import cross_validation

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
from keras import backend as K

from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.models import Sequential
from keras.layers import Activation
from keras.wrappers.scikit_learn import KerasRegressor

from keras.optimizers import SGD, adam, nadam, Adagrad
from keras.regularizers import l1,l2

from keras.callbacks import EarlyStopping, CSVLogger
from keras.losses import mean_squared_logarithmic_error


import os
import os.path
import sys
import argparse
import time
import csv

In [10]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18279197345194029353
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 104837939
locality {
  bus_id: 1
}
incarnation: 530339961367822436
physical_device_desc: "device: 0, name: GeForce GTX 970M, pci bus id: 0000:01:00.0"
]


In [11]:
def acc_loss(y_true, y_pred):
    
    # Subtract mean value from expansions of true and pred
    x_true = y_true[:,1:66]
    x_pred = y_pred[:,1:66]
    
    # Normalize each vector
    comp_true = tf.conj(x_true)
    norm_true = x_true / tf.sqrt(tf.reduce_sum(tf.multiply(x_true,comp_true)))
    
    comp_pred = tf.conj(x_pred)
    norm_pred = x_pred / tf.sqrt(tf.reduce_sum(tf.multiply(x_pred,comp_pred)))
    
    comp_p2 = tf.conj(norm_pred)
    acc = tf.real(tf.reduce_sum(tf.multiply(norm_true,comp_p2)))
    acc = -1.0 * acc * acc
    
    loss_mse = K.mean(K.square(y_pred-y_true))
    
    return acc + loss_mse

In [12]:
def load_all(input_path,output_path,test_input_path,test_output_path):
    
    input = loadmat(input_path)
    output = loadmat(output_path)
    
    #X = np.array(input['train_input_shore'])
    X = np.array(input['train_shore_input'])
    y = np.array(output['train_shore_output'])
    
    # Get dimensions of arrays
    print('Training Data Information \n')
    x_size = X.shape
    print('Input Array Shape',x_size)
    y_size = y.shape
    print ('Output Array Shape',y_size)
    
    test_input = loadmat(test_input_path)
    test_output = loadmat(test_output_path)
    
    #X_test = np.array(test_input['test_input_shore'])
    X_test = np.array(test_input['test_shore_input'])
    y_test = np.array(test_output['test_shore_output'])
    
    # Get dimensions of Test data
    print('Testing Data information \n')
    x_size = X_test.shape
    print('Test Input Shape',x_size)
    y_size = y_test.shape
    print('Test Output Shape',y_size)
    
    return X,y,X_test,y_test

In [13]:
def build_nn2():
    model = Sequential()
    # Input layer with dimension 1 and hidden layer i with 128 neurons.
    model.add(Dense(50, input_shape=(50,)))
    model.add(Dense(400))
    model.add(Activation("relu"))
    #model.add(Dropout(0.6))
    # Hidden layer j with 64 neurons plus activation layer.
    model.add(Dense(200))
    model.add(Activation("relu"))
    #model.add(Dropout(0.5))
    # Hidden layer k with 64 neurons.
    model.add(Dense(66))
    model.add(Activation("relu"))
    #model.add(Dropout(0.5))
    model.add(Dense(200))
    model.add(Activation("relu"))
    #model.add(Dropout(0.5))
    model.add(Dense(400))
    #model.add(Activation("relu"))
    # Output Layer.
    model.add(Dense(50))
 
    # Model is derived and compiled using mean square error as loss
    # function, accuracy as metric and gradient descent optimizer.
    model.compile(loss='mse', optimizer='RMSProp', metrics=['mse','mae'])
    model.summary()
    return model

In [14]:
# Originally number of epochs was set to 1000, currently at 10.
def train_nn(model, X, y, out_dir, val_size=0.1, n_epoch=1000):
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    csv_logger = CSVLogger(os.path.join(out_dir, 'results.csv'))

    model.fit(X, y, epochs=n_epoch, batch_size=1000, verbose=1, shuffle=True, validation_split=val_size, callbacks=[csv_logger])
    return model

In [15]:
def save_estimate(model, X, y, out_file, indices):
    #y_pred = model.predict(X)

    #y_pred = y_scaler.inverse_transform(y_pred)
    #y = y_scaler.inverse_transform(y)

    out_path = os.path.dirname(out_file)
    if not os.path.exists(out_path):
        os.makedirs(out_path)
     
    y_pred = model.predict(X)
    savemat(out_file, mdict={'out_pred': y_pred, 'out_true': y, 'indices': indices})

In [18]:
def save_test_set_prediction(model, out_file, X_test, y_test):
    
    # Get dimensions of arrays
    x_size = X_test.shape
    print('Hist 72: Input Array Shape',x_size)
    y_size = y_test.shape
    print ('Output Hist 72: Array Shape',y_size)
    
    # Make Predictions
    pred = model.predict(X_test)
    
    # If output path does not exist, create it
    out_path = os.path.dirname(out_file)
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    
    savemat(out_file, mdict={'out_pred': pred, 'out_true': y_test})

In [19]:
def save_vishabyte_predictions(model, out_file):
    input_file_path = r'D:\Users\Vishwesh\PycharmProjects\shore_mapmri\Data\vish_feed.mat'
    input_file_path = os.path.normpath(input_file_path)
    input_test = loadmat(input_file_path)
    X_f_t = np.array(input_test['vish_feed'])
    # Get dimensions of arrays
    x_size = X_f_t.shape
    print('Vishabyte: Input Array Shape',x_size)
    pred = model.predict(X_f_t)
    
    # If output path does not exist, create it
    out_path = os.path.dirname(out_file)
    if not os.path.exists(out_path):
        os.makedirs(out_path)
        
    # In vivo save matrix
    savemat(out_file, mdict={'out_pred': pred})

In [20]:
def main():

    #args = parse_args()
    work_dir = r'D:\Users\Vishwesh\PycharmProjects\shore_mapmri\dl_results'
    word_dir = os.path.normpath(work_dir)
    
    exp = 'Shore_input_Shore_output_v1'
    itr = 4 # Estimated from CV

    train_input_data_path = r'D:\Users\Vishwesh\PycharmProjects\shore_mapmri\Data\train_input_shore_v3.mat'
    train_output_data_path = r'D:\Users\Vishwesh\PycharmProjects\shore_mapmri\Data\train_output_shore_v3.mat'
    train_input_data_path = os.path.normpath(train_input_data_path)
    train_output_data_path = os.path.normpath(train_output_data_path)
    
    test_input_data_path = r'D:\Users\Vishwesh\PycharmProjects\shore_mapmri\Data\test_input_shore_v3.mat'
    test_output_data_path = r'D:\Users\Vishwesh\PycharmProjects\shore_mapmri\Data\test_output_shore_v3.mat'
    test_input_data_path = os.path.normpath(test_input_data_path)
    test_output_data_path = os.path.normpath(test_output_data_path)
    
    print ("Loading data")
    X, y, X_b_test, y_b_test = load_all(train_input_data_path,train_output_data_path,test_input_data_path,test_output_data_path)
    indices = np.array(range(X.shape[0]))+1

    out_start_dir = os.path.join(work_dir,exp)
    if not os.path.exists(out_start_dir):
        os.makedirs(out_start_dir)

    seed1 = 46
    seed2 = 23
    
    kf = KFold(n_splits=5, random_state=seed1,shuffle=True,)
    
    fold_num = 0
    model_D = build_nn2()
    
    
    for train, test in kf.split(X):
        # Set up training / testing data
        fold_num += 1
        X_train = X[train,:]
        y_train = y[train,:]
        X_test = X[test,:]
        y_test = y[test,:]
        indices_train = indices[train]
        indices_test = indices[test]
        #X_train, X_test, y_train, y_test, X_scaler, y_scaler = norm_data(X_train, X_test, y_train, y_test)



        # Want to submit this w/ 1000 different initializations
        np.random.seed(seed=seed2)

        # Deep NN
        print ("Training DNN with %d iterations, fold %d" % (itr, fold_num))
        out_dir_DNN = os.path.join(out_start_dir, str(fold_num))
        model_D = train_nn(model_D, X_train, y_train, out_dir_DNN, n_epoch=itr, val_size=0.1)

        print ("Saving training outputs")
        end_dir = os.path.join(out_start_dir, str(fold_num), 'training.csv')
        save_estimate(model_D, X_train, y_train, end_dir,indices_train)

        print ("Saving testing outputs")
        end_dir = os.path.join(out_start_dir, str(fold_num), 'testing.csv')
        save_estimate(model_D, X_test, y_test, end_dir,indices_test)

    #print ("Saving Vishabyte outputs")
    #end_dir = os.path.join(out_start_dir, str('Vishabyte_Test'), 'result_vol_invivo_vish_b2000.mat')
    #save_vishabyte_predictions(model_D, end_dir)

    print ("Saving Histology blind test outputs")
    end_dir = os.path.join(out_start_dir, str('Hist_Blind_72_Test'), 'result.mat')
    save_test_set_prediction(model_D, end_dir, X_b_test, y_b_test)
    
    print ("Saving Vishabyte Predictions")
    end_dir = os.path.join(out_start_dir, str('Vishabyte_Preds'), 'result.mat')
    save_vishabyte_predictions(model_D, end_dir)
                                   
    '''
    # Deep NN
    print ("Training DNN with %d iterations" % (itr))
    out_dir_DNN = os.path.join(out_start_dir, 'training_results')
    model_D = train_nn(model_D, X, y, out_dir_DNN, n_epoch=itr, val_size=0.2)

    print ("Saving testing outputs")
    end_dir = os.path.join(out_start_dir, 'testing_results')
    if not os.path.exists(end_dir):
        os.makedirs(end_dir)                           
    save_estimate(model_D, X_test, y_test, end_dir)

    kf = KFold(n_splits=5, random_state=seed1,shuffle=True,)

    fold_num = 0
    
    
    for train, test in kf.split(X):
        # Set up training / testing data
        fold_num += 1
        X_train = X[train,:]
        y_train = y[train,:]
        X_test = X[test,:]
        y_test = y[test,:]
        indices_train = indices[train]
        indices_test = indices[test]
        #X_train, X_test, y_train, y_test, X_scaler, y_scaler = norm_data(X_train, X_test, y_train, y_test)

        # Want to submit this w/ 1000 different initializations
        np.random.seed(seed=seed2)

        # Deep NN
        print ("Training DNN with %d iterations, fold %d" % (itr, fold_num))
        out_dir_DNN = os.path.join(out_start_dir, str(fold_num))
        model_D = train_nn(model_D, X_train, y_train, out_dir_DNN, n_epoch=itr, val_size=0.2)

        print ("Saving training outputs")
        end_dir = os.path.join(out_start_dir, str(fold_num), 'training.csv')
        save_estimate(model_D, X_train, y_train, end_dir,indices_train)

        print ("Saving testing outputs")
        end_dir = os.path.join(out_start_dir, str(fold_num), 'testing.csv')
        save_estimate(model_D, X_test, y_test, end_dir,indices_test)

    #print ("Saving Vishabyte outputs")
    #end_dir = os.path.join(out_start_dir, str('Vishabyte_Test'), 'result_vol_invivo_vish_b2000.mat')
    #save_vishabyte_predictions(model_D, end_dir)

    print ("Saving Histology blind test outputs")
    end_dir = os.path.join(out_start_dir, str('Hist_Blind_72_Test'), 'result_b6000_testing_10th_order.mat')
    save_test_set_prediction(model_D, end_dir)

    #print ("Saving Pair Wise Reprod Results")
    #end_dir_1 = os.path.join(out_start_dir, str('ISMRM_TS01_Test'), 'result_scan_a.mat')
    #end_dir_2 = os.path.join(out_start_dir, str('ISMRM_TS01_Test'), 'result_scan_b.mat')
    #pair_wise_reprod_check_2(model_D, end_dir_1, end_dir_2)
    
    print ("Saving Pair Wise Reprod Results on HCP")
    end_dir_1 = os.path.join(out_start_dir, str('HCP_Test'), 'result_scan_a.mat')
    end_dir_2 = os.path.join(out_start_dir, str('HCP_Test'), 'result_scan_b.mat')
    pair_wise_reprod_check_3(model_D, end_dir_1, end_dir_2)
    
    print ("Saving network for this fold")
    net_dir_DNN = (os.path.join(out_start_dir, 'nets'))
    if not os.path.exists(net_dir_DNN):
        os.makedirs(net_dir_DNN)
    net_name = os.path.join(net_dir_DNN, exp+"_model_fold"+str(fold_num)+".h5")
    model_D.save(net_name)

    t1 = time.time()
    total_time = t1-t0
    print (total_time)
'''

In [21]:
if __name__=='__main__':
    main()

Loading data
Training Data Information 

Input Array Shape (49995, 50)
Output Array Shape (49995, 50)
Testing Data information 

Test Input Shape (7272, 50)
Test Output Shape (7272, 50)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 400)               20400     
_________________________________________________________________
activation_1 (Activation)    (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               80200     
_________________________________________________________________
activation_2 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)       